In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import seaborn as sns
import statsmodels.api as sm
import sklearn

from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from sklearn import preprocessing
from initial_cleaning import initial_cleaning
from patsy import dmatrices

%matplotlib inline
sns.set_style('white')
pd.options.display.float_format = '{:.2f}'.format
plt.rcParams['figure.figsize'] = (20.0, 10.0)

/Users/jamilab/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
df = pd.DataFrame(initial_cleaning('online_sex_work.csv'))

/Users/jamilab/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2802: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


   User_ID Gender   Age Location  Verification Sexual_orientation  \
0 10386.00   male  34,6        A  Non_Verified         Homosexual   
1    14.00   male  32,2        J  Non_Verified       Heterosexual   
2 16721.00   male  33,6        K  Non_Verified       Heterosexual   
3 16957.00   male    34        H  Non_Verified       Heterosexual   
4 17125.00   male  39,5        B  Non_Verified       Heterosexual   

  Sexual_polarity Looking_for Points_Rank      Last_login Member_since  \
0          Switch         Men          50  before_10_days    17.9.2012   
1        Dominant       Women         518   before_1_days    1.11.2009   
2        Dominant       Women         150   before_3_days     1.4.2013   
3        Dominant       Women         114   before_4_days     8.4.2013   
4        Dominant       Women         497   before_5_days    14.4.2013   

  Number_of_Comments_in_public_forum Time_spent_chating_H:M  \
0                                 32                    0:2   
1             

In [3]:
# drop friends_id_list column from dataset

df = df.drop('Friends_ID_list', 1)

In [4]:
df.shape

(28768, 17)

In [5]:
# replace commas with periods & '???' with NaN

df['Age'] = df['Age'].replace(',','.', regex=True)
df['Age'] = df['Age'].replace('???', np.nan)

In [6]:
# fill missing values with mean age & convert to float

df['Age'] = df['Age'].astype(float)
df['Age'] = df['Age'].fillna(df['Age'].mean())

In [7]:
df['Sexual_orientation'].value_counts()

Heterosexual    20393
bisexual         4382
bicurious        2986
Homosexual       1005
Name: Sexual_orientation, dtype: int64

In [8]:
df['Sexual_orientation'] = df['Sexual_orientation'].fillna(df['Sexual_orientation'].mode())

In [9]:
df['Sexual_orientation'].describe()

count            28766
unique               4
top       Heterosexual
freq             20393
Name: Sexual_orientation, dtype: object

In [10]:
df['Gender'].describe()

count     28763
unique        2
top        male
freq      22338
Name: Gender, dtype: object

In [11]:
# list genders

df['Gender'].unique()

array(['male', 'female', nan], dtype=object)

In [12]:
df['Gender'] = df['Gender'].fillna(df['Gender'].mode())

In [13]:
df['Gender'].describe()

count     28763
unique        2
top        male
freq      22338
Name: Gender, dtype: object

In [14]:
df['Gender'].value_counts()

male      22338
female     6425
Name: Gender, dtype: int64

In [15]:
df['Looking_for'].unique()

array(['Men', 'Women', 'Nobody_but_maybe', 'Nobody', 'Men_and_Women', nan], dtype=object)

In [16]:
# remove commas, spaces & the letter a from numbers in points rank column

df['Points_Rank'] = df['Points_Rank'].replace(',','.', regex=True)
df['Points_Rank'] = df['Points_Rank'].replace(' ','', regex=True)
df['Points_Rank'] = df['Points_Rank'].replace('a',np.nan)

In [17]:
# convert points rank to float & describe data

df['Points_Rank'] = df['Points_Rank'].astype(float)
df['Points_Rank'].describe()

count   28560.00
mean        9.62
std        38.64
min      -650.00
25%         0.00
50%         0.00
75%        11.00
max      1626.00
Name: Points_Rank, dtype: float64

In [18]:
# create feature to make gender categorical

df['gendercat'] = np.where(df['Gender'] == 'female', 1, 0)

In [19]:
# create interactions for homosexual male, homosexual female & dominant female

df['homo_male'] = (df['Gender'] == 'male').astype(int) * (df['Sexual_orientation'] == 'Homosexual').astype(float)
df['homo_fem'] = (df['Gender'] == 'female').astype(int) * (df['Sexual_orientation'] == 'Homosexual').astype(float)
df['fem_dom'] = (df['Gender'] == 'female').astype(int) * (df['Sexual_polarity'] == 'Dominant').astype(float)

In [20]:
# create features for sexual orientation

df['homo'] = np.where(df['Sexual_orientation'] == 'Homosexual', 1, 0).astype(float)
df['hetero'] = np.where(df['Sexual_orientation'] == 'Heterosexual', 1, 0).astype(float)
df['bisexual'] = np.where(df['Sexual_orientation'] == 'bisexual', 1, 0).astype(float)
df['bicurious'] = np.where(df['Sexual_orientation'] == 'bicurious', 1, 0).astype(float)

In [21]:
# create features for sexual polarity

df['dom'] = np.where(df['Sexual_polarity'] == 'Dominant', 1, 0)
df['sub'] = np.where(df['Sexual_polarity'] == 'Submisive', 1, 0)
df['switch'] = np.where(df['Sexual_polarity'] == 'Switch', 1, 0)

In [22]:
df.head(10)

User_ID  Gender   Age Location  Verification Sexual_orientation  \
0 10386.00    male 34.60        A  Non_Verified         Homosexual   
1    14.00    male 32.20        J  Non_Verified       Heterosexual   
2 16721.00    male 33.60        K  Non_Verified       Heterosexual   
3 16957.00    male 34.00        H  Non_Verified       Heterosexual   
4 17125.00    male 39.50        B  Non_Verified       Heterosexual   
5 17220.00  female 47.80        A  Non_Verified       Heterosexual   
6  1726.00    male 36.70        B  Non_Verified          bicurious   
7  1750.00    male 51.00        L  Non_Verified       Heterosexual   
8 18162.00  female 29.50        A  Non_Verified          bicurious   
9 18820.00    male 28.00        J  Non_Verified       Heterosexual   

  Sexual_polarity       Looking_for  Points_Rank      Last_login   ...    \
0          Switch               Men        50.00  before_10_days   ...     
1        Dominant             Women       518.00   before_1_days   ...     
2        Dominant             Women       150.00   before_3_days   ...     
3        Dominant             Women       114.00   before_4_days   ...     
4        Dominant             Women       497.00   before_5_days   ...     
5        Dominant  Nobody_but_maybe       127.00   before_1_days   ...     
6       Submisive             Women        18.00   before_1_days   ...     
7        Dominant            Nobody        38.00   before_9_days   ...     
8       Submisive  Nobody_but_maybe       150.00   before_4_days   ...     
9          Switch             Women       -26.00   before_1_days   ...     

  homo_male homo_fem fem_dom  homo  hetero  bisexual bicurious  dom  sub  \
0      1.00     0.00    0.00  1.00    0.00      0.00      0.00    0    0   
1      0.00     0.00    0.00  0.00    1.00      0.00      0.00    1    0   
2      0.00     0.00    0.00  0.00    1.00      0.00      0.00    1    0   
3      0.00     0.00    0.00  0.00    1.00      0.00      0.00    1    0   
4      0.00     0.00    0.00  0.00    1.00      0.00      0.00    1    0   
5      0.00     0.00    1.00  0.00    1.00      0.00      0.00    1    0   
6      0.00     0.00    0.00  0.00    0.00      0.00      1.00    0    1   
7      0.00     0.00    0.00  0.00    1.00      0.00      0.00    1    0   
8      0.00     0.00    0.00  0.00    0.00      0.00      1.00    0    1   
9      0.00     0.00    0.00  0.00    1.00      0.00      0.00    0    0   

   switch  
0       1  
1       0  
2       0  
3       0  
4       0  
5       0  
6       0  
7       0  
8       0  
9       1  

[10 rows x 28 columns]

In [23]:
df['Risk'].unique()

array(['No_risk', 'High_risk', 'unknown_risk', nan], dtype=object)

In [24]:
df['Verification'].unique()

array(['Non_Verified', 'Verified', 'Administrator', nan], dtype=object)

In [25]:
df['Risk'] = np.where(df['Risk'] == 'High_risk', 1, 0)
df['Verification'] = np.where(df['Verification'] == 'Verified', 1, 0)

In [26]:
df.columns

Index(['User_ID', 'Gender', 'Age', 'Location', 'Verification',
       'Sexual_orientation', 'Sexual_polarity', 'Looking_for', 'Points_Rank',
       'Last_login', 'Member_since', 'Number_of_Comments_in_public_forum',
       'Time_spent_chating_H:M', 'Number_of_advertisments_posted',
       'Number_of_offline_meetings_attended', 'Profile_pictures', 'Risk',
       'gendercat', 'homo_male', 'homo_fem', 'fem_dom', 'homo', 'hetero',
       'bisexual', 'bicurious', 'dom', 'sub', 'switch'],
      dtype='object')

In [27]:
df = df.drop('Time_spent_chating_H:M', 1)
df = df.drop('Number_of_advertisments_posted', 1)
df = df.drop('Number_of_Comments_in_public_forum', 1)
df = df.drop('Number_of_offline_meetings_attended', 1)
df = df.drop('Member_since', 1)
df = df.drop('Last_login', 1)
df = df.drop('Location', 1)
df = df.drop('Looking_for', 1)
df = df.drop('Sexual_polarity', 1)
df = df.drop('Sexual_orientation', 1)
df = df.drop('Gender', 1)
df = df.drop('User_ID', 1)
df = df.drop('Profile_pictures', 1)
df = df.drop('Age', 1)
df = df.drop('Points_Rank', 1)

# Fitting a binary logistic model using statsmodels

**FYI: I couldn't get the prediction table to work in the 2nd cell.**

In [28]:
# Declare predictors.
X_statsmod = df[['hetero', 'homo', 'bisexual', 'bicurious']]

# The Statsmodels formulation requires a column with constant value 1 that
# will act as the intercept.
X_statsmod['intercept'] = 1 

# Declare and fit the model.
logit = sm.Logit(df['gendercat'], X_statsmod)
result = logit.fit()

# Lots of information about the model and its coefficients, but the
# accuracy rate for predictions is missing.
print(result.summary())

/Users/jamilab/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


         Current function value: 0.513634
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:              gendercat   No. Observations:                28768
Model:                          Logit   Df Residuals:                    28763
Method:                           MLE   Df Model:                            4
Date:                Sun, 13 May 2018   Pseudo R-squ.:                 0.03289
Time:                        19:45:38   Log-Likelihood:                -14776.
converged:                      False   LL-Null:                       -15279.
                                        LLR p-value:                2.936e-216
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
hetero        14.5560   2200.406      0.007      0.995   -4298.161    4327.273
homo          14.2895   2200.406      0.006      0.995   -4298.42

/Users/jamilab/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [29]:
# Calculate accuracy. First, get probability that each row will be admitted.
pred_statsmod = result.predict(X_statsmod)

# Code female as 1 if probability is greater than .5.
pred_y_statsmod = np.where(pred_statsmod < .5, 0, 1)

# Accuracy table.
table = pd.crosstab(df['gendercat'], pred_y_statsmod)

print('\n Accuracy by gender')
print(table)
print('\n Percentage accuracy')
print((table.iloc[0,0] + table.iloc[1,1]) / (table.sum().sum()))


 Accuracy by gender
col_0          0
gendercat       
0          22343
1           6425

 Percentage accuracy


IndexError: single positional indexer is out-of-bounds

# Fitting a binary logistic model using SKLearn

In [30]:
# Declare a logistic regression classifier.
# Parameter regularization coefficient C described above.
lr = LogisticRegression(C=1e9)
y = df['gendercat']
X = df[['homo', 'hetero', 'bisexual', 'bicurious']]

# Fit the model.
fit = lr.fit(X, y)

# Display.
print('Coefficients')
print(fit.coef_)
print(fit.intercept_)
pred_y_sklearn = lr.predict(X)

print('\n Accuracy by sexual orientation')
print(pd.crosstab(pred_y_sklearn, y))

print('\n Percentage accuracy')
print(lr.score(X, y))

Coefficients
[[-0.46887022 -0.2019435   0.76195304  0.73804582]]
[-1.32802468]

 Accuracy by sexual orientation
gendercat      0     1
row_0                 
0          22343  6425

 Percentage accuracy
0.776661568409


# Ridge Regression

In [31]:
# Define the training and test sizes.
trainsize = int(df.shape[0] / 2)
df_test = df.iloc[trainsize:, :].copy()
df_train = df.iloc[:trainsize, :].copy()

# Set up the regression model to predict defaults using all other variables as features.
regr1 = linear_model.LinearRegression()
Y_train = df_train['gendercat'].values.reshape(-1, 1)
X_train = df_train
regr1.fit(X_train, Y_train)
print('\nR-squared:')
print(regr1.score(X_train, Y_train))

#Store the parameter estimates.
origparams = np.append(regr1.coef_, regr1.intercept_)
print(origparams)


R-squared:
1.0
[ -2.75116220e-16   4.71844785e-15   1.00000000e+00   9.57260910e-02
   9.57260910e-02  -2.16493490e-15   4.33964768e-01   5.29690859e-01
   5.29690859e-01   5.29690859e-01  -5.29690859e-01  -5.29690859e-01
  -5.29690859e-01  -1.06858966e-14]


In [32]:
# Test the model
Y_test = df_test['gendercat'].values.reshape(-1, 1)
X_test = df_test
print('\nR-squared:')
print(regr1.score(X_test, Y_test))


R-squared:
1.0


In [33]:
ridgeregr = linear_model.Ridge(alpha=10, fit_intercept=False) 
ridgeregr.fit(X_train, Y_train)
print(ridgeregr.score(X_train, Y_train))

# Store the new parameter estimates for the same features.
newparams = np.append(
    ridgeregr.coef_[0,0:(len(origparams)-1)],
    ridgeregr.intercept_)
print(newparams)

0.99997584942
[  6.19854530e-04  -9.35436483e-04   9.94101295e-01  -1.64963146e-03
   2.28146734e-03   5.55715115e-03   6.31835879e-04   2.15621105e-04
   5.82704299e-04   6.55550118e-04  -2.17066179e-04   1.70012761e-03
   6.02649966e-04   0.00000000e+00]


# Lasso Regression  

**All I could get it do to was not fail. Everything else is 0. 🤷🏾‍♀️🤷🏾‍♀️**

In [34]:
LY_train = df_train['gendercat'].values.reshape(-1, 1)
LX_train = df_train

# Small number of parameters.
lass = linear_model.Lasso(alpha=35)
lassfit = lass.fit(LX_train, LY_train)
print('R²:')
print(lass.score(X_train, Y_train))

lassparams = np.append(lassfit.coef_, lassfit.intercept_)
print('\nParameter estimates:')
print(lassparams)

R²:
0.0

Parameter estimates:
[ 0.        -0.         0.        -0.         0.         0.        -0.        -0.
  0.         0.        -0.         0.        -0.         0.2128059]


In [35]:
print(lass.score(X_train, Y_train))

0.0
